In [1]:
import os
import sys
from datetime import datetime
from typing import Optional, Union, Dict

import requests
import pandas as pd

from onchain.utils import *

In [2]:
# Setup Alchemy API and Session for makng requests

# os.environ['ALCHEMY_API_KEY'] = ''
alchemy_request_url = get_alchemy_request_url()
session = requests.Session()

ALCHEMY_API_KEY env variable not set! Trying from config.py.
Successfully found ALCHEMY_API_KEY!


In [3]:
# Defining addresses that we want to query
input_addresses = [
    "0xBE5F037E9bDfeA1E5c3c815Eb4aDeB1D9AB0137B", 
    "0xB033F13BB4F1cAF484AB5c090F22bE425b2146B3",
]

In [4]:
# Getting the latest block for consistency in case
# the network moves to the next block while the script is running
latest_block = get_latest_block(alchemy_request_url, session)
latest_block = latest_block if latest_block else 'latest'

Latest block as of time 2022-03-17 17:01:54.656514 is 0xdbd5aa


In [5]:
balance_output = {}
transaction_history_output = {}
asset_count_summary_output = {}

In [6]:
# Main Driver code
for address in input_addresses:
    # Validate input addresses. If an address is invalid, 
    # skip to the next one that the user specified
    if not validate_input_address(address):
        print(f"Input string '{address}' is of the wrong format to be valid address! Skipping.")
        continue

    # If the address is valid, get it's ether balance at the latest block
    balance = get_balance(address, alchemy_request_url, session, block_num=latest_block)
    
    # Only continue with remainder of the logic if get_balance executes successfully
    if balance is None:
        print(f"get_balance() request failed for address {address}!")
        print("Continuing to next address")
        continue
        
    # Store the balance that we've computed
    balance_output[address] = balance

    # Get the transactions that were sent FROM the address
    transfers_from = get_transaction_history(
        address,
        alchemy_request_url,
        True,
        session,
        latest_block,
    )
    
    # Get the transactions that were sent TO the address
    transfers_to = get_transaction_history(
        address,
        alchemy_request_url,
        False,
        session,
        latest_block,
    )
    
    if transfers_to and not transfers_from:
        # If an address has been funded, but has made no "out" transactions
        
        # Convert the block number from hex to integer. Sort by block # descending.
        df_transaction_history['blockNum'] = df_transaction_history['blockNum'].apply(lambda x: int(x, 16))
        df_transaction_history.sort_values(by = 'blockNum', ascending=False, inplace=True)
        
        # Dedupe transactions on tx hash
        df_transaction_history = dedupe_transaction_history(pd.DataFrame.from_dict(transfers_from)).reset_index(drop=True)

        # Get the current balance for each asset.
        df_asset_count = df_from.groupby('asset').sum('value') 
        df_asset_count.fillna(0, inplace=True)
    elif transfers_from and transfers_to:
        # If an address both has "from" and "to" transactions
        df_from = pd.DataFrame.from_dict(transfers_from)
        df_to = pd.DataFrame.from_dict(transfers_to)

        # Concatenate the "from" and "to" transactions into one dataframe
        df_transaction_history = pd.concat([df_from, df_to])
        
        # Convert the block number from hex to integer. Sort by block # descending.
        df_transaction_history['blockNum'] = df_transaction_history['blockNum'].apply(lambda x: int(x, 16))
        df_transaction_history.sort_values(by = 'blockNum', ascending=False, inplace=True)

        # Dedupe transactions on tx hash
        df_transaction_history = dedupe_transaction_history(df_transaction_history).reset_index(drop=True)

        # Get the current balance for each asset for the "from" and "to" sets separately
        df_from_asset_count = df_from.groupby('asset').sum('value')
        df_to_asset_count = df_to.groupby('asset').sum('value')

        # Join these results together and compute the current balance
        # by subtracting the "from" values from the "to" values
        df_asset_count = df_to_asset_count.merge(df_from_asset_count, on = 'asset', how='outer')
        df_asset_count.columns = ['to_balance', 'from_balance']
        df_asset_count.fillna(0, inplace=True)

        df_asset_count['current_balance'] = df_asset_count['to_balance'] - df_asset_count['from_balance']
        df_asset_count.drop(columns = ["to_balance", "from_balance"], inplace=True)
    else:
        # If both API calls return None or empty list
        # If the address doesn't have any "to" transactions, can't happen
        print(f"Transaction history parsing for address {address} failed! Continuing.")
        continue
        
    # Drop NFT metadata columns. 
    # ERC721 tx will stil appear in the output dataframe, but the count
    # will not be correct without doing some additional parsing.
    df_transaction_history = df_transaction_history.drop(['erc721TokenId', 'erc1155Metadata', 'tokenId'], axis = 1)

    # Remove tokens that the address may have held in the past but now has a 0 balance.
    # Remove exotic "migrate" transactions (see note about gOHM in README)
    df_asset_count = df_asset_count.loc[(df_asset_count > 0).any(axis=1)]

    # Set ETH current balance to previously computed balance value,
    # necessary without having to compute gas for all transactions
    df_asset_count[df_asset_count.index == 'ETH'] = balance_output[address]
              
        
    # Store the transaction history and asset count summary that we've computed
    transaction_history_output[address] = df_transaction_history
    asset_count_summary_output[address] = df_asset_count

In [8]:
# Print output
if len(asset_count_summary_output) == 0:
    print('No summaries available!')
else:
    for valid_address in asset_count_summary_output:
        print(f"Printing transaction summary for address: {address} as of block {latest_block}\n")
        print(f"It's current balance is \n{balance_output[valid_address]} ether\n")
        print(f"It's current ERC20 token (and ETH) balance is {asset_count_summary_output[valid_address]}\n")
        print(f"This address had {len(transaction_history_output[valid_address])} transactions in total (completed + failed)\n")
        print(f"Here are the 5 most recent transactions:\n{transaction_history_output[valid_address].head()}\n")
        print("-"*90)

Printing transaction summary for address: 0xB033F13BB4F1cAF484AB5c090F22bE425b2146B3 as of block 0xdbd5aa

It's current balance is 
0.009581141935878367 ether

It's current ERC20 token (and ETH) balance is        current_balance
asset                 
DAI           0.837671
ETH           0.009581
FREE      48018.998006
GOHM          0.014116
GRT         330.183524
MATIC       414.304437
OHM           0.108188
SPICE     85000.000000
ZURR          2.000000

This address had 57 transactions in total (completed + failed)

Here are the 5 most recent transactions:
   blockNum                                               hash  \
0  14312082  0xc7c9131f8a83e69741bcf3ddaebef574da6b9dbc9aed...   
1  14219676  0xd12e6cb81f2d0f8e02de977328e8ae5ee21b0cfce843...   
2  14186046  0xbd544bfaf1826df1dea81c0d312c09cfebe832494381...   
3  14073073  0xec134f1f0b34b78c062136b87a2fb8e35218a711a77b...   
4  14073026  0x720f329a3a060bfaefff6cf5f90f979f643d1a8c5101...   

                                      

## Examples from Etherscan
https://etherscan.io/address/0xbe5f037e9bdfea1e5c3c815eb4adeb1d9ab0137b
https://etherscan.io/address/0xB033F13BB4F1cAF484AB5c090F22bE425b2146B3